## Урок 4. Сверточные нейронные сети

## Практическое задание

Вариант 1. (простой)

- обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг  (conv-pool)-(conv-pool)-...) на датасете fashion-mnist или cifar10
- оценить рост точности при увеличении ширины сети (больше ядер)
- оценить рост точности при увеличении глубины сети (больше слоев)
- (опциционально)сравнить с точностью полносвязной сети для этой выборки
    </li>

Вариант 2. (сложный)
- реализовать нейронную сеть или стек из сверток (Concatenate) на сifar10.
- оценить рост точности при увеличении ширины сети (больше ядер), больше нитей.
- оценить рост точности при увеличении глубины сети (больше слоев)
    </li>
</ol>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
from __future__ import print_function
import tensorflow.keras as keras 
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import models, layers,  losses
from tensorflow.keras.losses import categorical_crossentropy
import os

#### Загрузка датасета

In [ ]:
classes=['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

In [ ]:
# установка параметров нейросети
batch_size = 32
num_classes = 10
data_augmentation = False
epochs = 30
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model_h5'

# разделение тренировочной и тестовой выбоки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape', x_train.shape)
print(x_train.shape[0], 'тренировочный пример')
print(x_test.shape[0], 'тестовый пример')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

170498071/170498071 [==============================] - 2s 0us/step
x_train shape (50000, 32, 32, 3)
50000 тренировочный пример
10000 тестовый пример


In [ ]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout( 0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# инициализация RMSprop optimizer
#opt = tensorflow.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])



if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test), 
              verbose=0,
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=5, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test), verbose=0)

# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Не используется data augmentation


сохранить обученную модель как /content/saved_models/keras_cifar10_trained_model_h5 
Test loss: 0.6781342625617981
Test accuracy: 0.7659000158309937


### Оценка влияния увеличения ширины сети

In [ ]:
result_width = []

In [ ]:
def models_alexnet(width, epochs=epochs):
      # конфигурирование слоев нейросети
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32 + width, (3, 3), padding='same',
                    input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32 + width, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout( 0.25))

    model.add(Conv2D(64 + width, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64 + width, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # инициализация RMSprop optimizer
    #opt = tensorflow.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer='SGD',
                  metrics=['accuracy'])



    if not data_augmentation:
        print('Не используется data augmentation')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test), 
                  verbose=0,
                  shuffle=True)
    else:
        print('Использование data augmentation в реальном времени')
        # Препроцессинг и data augmentation в реальном времени:
        datagen = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False, 
            zca_epsilon=1e-06, 
            rotation_range=5, 
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0., 
            zoom_range=0., 
            channel_shift_range=0.,
            fill_mode='nearest',
            cval=0.,
            horizontal_flip=True,
            vertical_flip=False,
            rescale=None,
            preprocessing_function=None,
            data_format=None,
            validation_split=0.0)

        # запуск data augmentation через fit
        #datagen.fit(x_train)

        # запуск data augmentation через fit_generator
        model.fit_generator(datagen.flow(x_train, y_train,
                                        batch_size=batch_size),
                            epochs=epochs,
                            validation_data=(x_test, y_test), verbose=0)

    # проверка работы обученной модели
    [loss, accuracy] = model.evaluate(x_test, y_test)
    result_width.append([width, loss, accuracy])
    return result_width

In [ ]:
models_alexnet(8, epochs=30)

Не используется data augmentation
313/313 [==============================] - 1s 4ms/step - loss: 0.6263 - accuracy: 0.7854


[[8, 0.6262965798377991, 0.7853999733924866]]

In [ ]:
models_alexnet(16, epochs=30)

Не используется data augmentation
313/313 [==============================] - 1s 4ms/step - loss: 0.6202 - accuracy: 0.7823


[[8, 0.6262965798377991, 0.7853999733924866],
 [16, 0.6202129125595093, 0.7822999954223633]]

In [ ]:
models_alexnet(32, epochs=30)

Не используется data augmentation
313/313 [==============================] - 1s 4ms/step - loss: 0.6228 - accuracy: 0.7841


[[8, 0.6262965798377991, 0.7853999733924866],
 [16, 0.6202129125595093, 0.7822999954223633],
 [32, 0.62276691198349, 0.7840999960899353]]

In [ ]:
models_alexnet(64, epochs=30)

Не используется data augmentation
313/313 [==============================] - 2s 5ms/step - loss: 0.6114 - accuracy: 0.7965


[[8, 0.6262965798377991, 0.7853999733924866],
 [16, 0.6202129125595093, 0.7822999954223633],
 [32, 0.62276691198349, 0.7840999960899353],
 [64, 0.6114149689674377, 0.796500027179718]]

In [ ]:
models_alexnet(128, epochs=30)

Не используется data augmentation
313/313 [==============================] - 2s 7ms/step - loss: 0.6328 - accuracy: 0.7983


[[8, 0.6262965798377991, 0.7853999733924866],
 [16, 0.6202129125595093, 0.7822999954223633],
 [32, 0.62276691198349, 0.7840999960899353],
 [64, 0.6114149689674377, 0.796500027179718],
 [128, 0.6328408122062683, 0.79830002784729]]

In [ ]:
models_alexnet(192, epochs=30)

Не используется data augmentation
313/313 [==============================] - 3s 9ms/step - loss: 0.6868 - accuracy: 0.7862


[[8, 0.6262965798377991, 0.7853999733924866],
 [16, 0.6202129125595093, 0.7822999954223633],
 [32, 0.62276691198349, 0.7840999960899353],
 [64, 0.6114149689674377, 0.796500027179718],
 [128, 0.6328408122062683, 0.79830002784729],
 [192, 0.6868249773979187, 0.7861999869346619]]

In [ ]:
pd.DataFrame(result_width, columns=['width',
                              'loss', 'accuracy'
                              ]).\
                              sort_values('accuracy',ascending=False).head(4)

width      loss  accuracy
4    128  0.632841    0.7983
3     64  0.611415    0.7965
5    192  0.686825    0.7862
0      8  0.626297    0.7854

In [ ]:
result_depth = []

In [ ]:
def models_alexnet(width, depth, epochs=epochs):
      # конфигурирование слоев нейросети
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32 + width, (3, 3), padding='same',
                    input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32 + width, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout( 0.25))

    model.add(Conv2D(64 + width, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64 + width, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    
    for i in range(depth):
        model.add(Conv2D(width, (3, 3), padding='same'))
        model.add(Activation('relu'))    

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # инициализация RMSprop optimizer
    #opt = tensorflow.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer='SGD',
                  metrics=['accuracy'])



    if not data_augmentation:
        print('Не используется data augmentation')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test), 
                  verbose=0,
                  shuffle=True)
    else:
        print('Использование data augmentation в реальном времени')
        # Препроцессинг и data augmentation в реальном времени:
        datagen = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False, 
            zca_epsilon=1e-06, 
            rotation_range=5, 
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0., 
            zoom_range=0., 
            channel_shift_range=0.,
            fill_mode='nearest',
            cval=0.,
            horizontal_flip=True,
            vertical_flip=False,
            rescale=None,
            preprocessing_function=None,
            data_format=None,
            validation_split=0.0)

        # запуск data augmentation через fit
        #datagen.fit(x_train)

        # запуск data augmentation через fit_generator
        model.fit_generator(datagen.flow(x_train, y_train,
                                        batch_size=batch_size),
                            epochs=epochs,
                            validation_data=(x_test, y_test), verbose=0)

    # проверка работы обученной модели
    [loss, accuracy] = model.evaluate(x_test, y_test)
    result_depth.append([width, depth, loss, accuracy])
    return result_depth

In [ ]:
models_alexnet(width=64, depth=2, epochs=30)

Не используется data augmentation
313/313 [==============================] - 2s 5ms/step - loss: 0.5770 - accuracy: 0.8042


[[64, 2, 0.5769685506820679, 0.8041999936103821]]

In [ ]:
models_alexnet(width=64, depth=3, epochs=30)

Не используется data augmentation
313/313 [==============================] - 2s 5ms/step - loss: 0.5816 - accuracy: 0.8088


[[64, 2, 0.5769685506820679, 0.8041999936103821],
 [64, 3, 0.5815987586975098, 0.8087999820709229]]

In [ ]:
models_alexnet(width=64, depth=4, epochs=30)

Не используется data augmentation
313/313 [==============================] - 2s 5ms/step - loss: 0.6201 - accuracy: 0.7965


[[64, 2, 0.5769685506820679, 0.8041999936103821],
 [64, 3, 0.5815987586975098, 0.8087999820709229],
 [64, 4, 0.6200562715530396, 0.796500027179718]]

In [ ]:
models_alexnet(width=128, depth=2, epochs=30)

Не используется data augmentation
313/313 [==============================] - 2s 7ms/step - loss: 0.6165 - accuracy: 0.8083


[[64, 2, 0.5769685506820679, 0.8041999936103821],
 [64, 3, 0.5815987586975098, 0.8087999820709229],
 [64, 4, 0.6200562715530396, 0.796500027179718],
 [8, 4, 0.8362809419631958, 0.7113000154495239],
 [8, 2, 0.7348443865776062, 0.7477999925613403],
 [8, 3, 0.7774717807769775, 0.7301999926567078],
 [128, 2, 0.6164664030075073, 0.8083000183105469]]

In [ ]:
models_alexnet(width=128, depth=3, epochs=30)

Не используется data augmentation
313/313 [==============================] - 2s 7ms/step - loss: 0.6194 - accuracy: 0.7996


[[64, 2, 0.5769685506820679, 0.8041999936103821],
 [64, 3, 0.5815987586975098, 0.8087999820709229],
 [64, 4, 0.6200562715530396, 0.796500027179718],
 [8, 4, 0.8362809419631958, 0.7113000154495239],
 [8, 2, 0.7348443865776062, 0.7477999925613403],
 [8, 3, 0.7774717807769775, 0.7301999926567078],
 [128, 2, 0.6164664030075073, 0.8083000183105469],
 [128, 3, 0.6193881630897522, 0.7996000051498413]]

In [ ]:
models_alexnet(width=128, depth=4, epochs=30)

Не используется data augmentation
313/313 [==============================] - 2s 7ms/step - loss: 0.5901 - accuracy: 0.8126


[[64, 2, 0.5769685506820679, 0.8041999936103821],
 [64, 3, 0.5815987586975098, 0.8087999820709229],
 [64, 4, 0.6200562715530396, 0.796500027179718],
 [8, 4, 0.8362809419631958, 0.7113000154495239],
 [8, 2, 0.7348443865776062, 0.7477999925613403],
 [8, 3, 0.7774717807769775, 0.7301999926567078],
 [128, 2, 0.6164664030075073, 0.8083000183105469],
 [128, 3, 0.6193881630897522, 0.7996000051498413],
 [128, 4, 0.5901492238044739, 0.8126000165939331]]

In [ ]:
models_alexnet(width=8, depth=2, epochs=30)

Не используется data augmentation
313/313 [==============================] - 1s 4ms/step - loss: 0.7348 - accuracy: 0.7478


[[64, 2, 0.5769685506820679, 0.8041999936103821],
 [64, 3, 0.5815987586975098, 0.8087999820709229],
 [64, 4, 0.6200562715530396, 0.796500027179718],
 [8, 4, 0.8362809419631958, 0.7113000154495239],
 [8, 2, 0.7348443865776062, 0.7477999925613403]]

In [ ]:
models_alexnet(width=8, depth=3, epochs=30)

Не используется data augmentation
313/313 [==============================] - 1s 4ms/step - loss: 0.7775 - accuracy: 0.7302


[[64, 2, 0.5769685506820679, 0.8041999936103821],
 [64, 3, 0.5815987586975098, 0.8087999820709229],
 [64, 4, 0.6200562715530396, 0.796500027179718],
 [8, 4, 0.8362809419631958, 0.7113000154495239],
 [8, 2, 0.7348443865776062, 0.7477999925613403],
 [8, 3, 0.7774717807769775, 0.7301999926567078]]

In [ ]:
models_alexnet(width=8, depth=4, epochs=30)

Не используется data augmentation
313/313 [==============================] - 1s 4ms/step - loss: 0.8363 - accuracy: 0.7113


[[64, 2, 0.5769685506820679, 0.8041999936103821],
 [64, 3, 0.5815987586975098, 0.8087999820709229],
 [64, 4, 0.6200562715530396, 0.796500027179718],
 [8, 4, 0.8362809419631958, 0.7113000154495239]]

In [ ]:
# Оценка влияния увеличения глубины сети на тестовой выборке
pd.DataFrame(result_depth, columns=['width','depth',
                              'loss', 'accuracy'
                              ]).\
                              sort_values('accuracy',ascending=False).head(6)

width  depth      loss  accuracy
8    128      4  0.590149    0.8126
1     64      3  0.581599    0.8088
6    128      2  0.616466    0.8083
0     64      2  0.576969    0.8042
7    128      3  0.619388    0.7996
2     64      4  0.620056    0.7965

Итог:
 - Увеличивая ширину сети(количества ядер) повышает качество.
 - Увеличение глубины(конв. слоев) может влиять по-разному на точность.

